In [370]:
import numpy as np
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
import json
import requests
import bs4
import ast

# CisBP

In [19]:
fp_cisbp = os.path.join('data', 'CisBP_C2H2_ZFs', 'PWM.txt')

def CisBP_to_df(fp):
    """
    fp: file path to desired file
    """
    ZF_entries = open(fp).read().split('\n\n\n')
    ZF_entries = [ZF_entry.replace('\t', ' ').split('\n') for ZF_entry in ZF_entries]
    ZF_entries = process_ZF_entries(ZF_entries)
    
    return pd.DataFrame(ZF_entries)

def process_ZF_entries(ZF_entries):
    output = []
    for lst in ZF_entries:
        temp = [elem.rpartition(' ') for elem in lst[:6]]
        if len(temp) > 4:
            del temp[4]
        d = {}
        for tup in temp:
            d[tup[0]] = tup[2] # tup[1] is just a space
        
        PWM = lst[7:]
        PWM = [column.partition(' ')[2] for column in PWM]
        d['PWM'] = transpose_PWM(PWM)
        
        output.append(d)
    return output

def transpose_PWM(PWM):
    rows = [row.split() for row in PWM]
    transpose = [[0 for _ in range(len(PWM))] for _ in range(4)]
    for i in range(4):
        transpose[i] = [float(row[i]) for row in rows]
    return np.array(transpose)

def is_empty_PWM(PWM):
    return PWM.tolist() != np.array([[],[],[],[]]).tolist()

test = CisBP_to_df(fp_cisbp).drop(columns=['']).iloc[:-1]
test['has_PWM'] = test['PWM'].apply(is_empty_PWM)
cleaned_cisbp = test[test['has_PWM'] == True].drop(columns=['has_PWM'])

### Merge CisBP with UniProt_ZF

In [128]:
fp_new_uniprot_zf = os.path.join('data', 'Transcription Factors (UniProt).xlsx')
uniprot_zf = pd.read_excel(fp_new_uniprot_zf)

#fp_old_uniprot_zf = os.path.join('data', 'UniProt_ZF.csv')
#uniprot_zf = pd.read_csv(open(fp_old_uniprot_zf)).drop(columns=['Unnamed: 0', 'DNA Binding Sequence (to be added)'])
uniprot_zf = uniprot_zf.rename(columns={'Entry': 'UniProt ID'})
uniprot_zf = uniprot_zf[uniprot_zf['Gene names'].notnull()]

In [123]:
def uniprot_gene_names_contains(name):
    """
    Match TF Name to Gene names (same column in both sheets)
    """
    name = name.lower()
    lst = []
    for i, entry in enumerate(uniprot_zf['Gene names'].str.lower()):
        if re.search('(^| )' + name + '($| )', entry) != None: 
            # the above is some regex. ^ means start of line, $ means end of line, | means or. 
            lst.append({'index': i, 'name': entry})
            # index is the row index (access by iloc) in uniprot_zf to which the match corresponds
    return lst

In [124]:
cleaned_cisbp = cleaned_cisbp.assign(possible_match=cleaned_cisbp['TF Name'].apply(uniprot_gene_names_contains))
cleaned_cisbp = cleaned_cisbp.assign(num_possible_match=cleaned_cisbp['possible_match'].apply(lambda l: len(l)))
matched_cisbp = cleaned_cisbp[cleaned_cisbp['num_possible_match'] != 0]

In [125]:
def filter_by_organism(ser):
    """
    Run new_df = matched_cisbp.apply(filter_by_organism), which should take the list in possible_match and 
    eliminate entries that correspond to a TF that doesn't belong to the Species column in matched_cisbp
    """
    lst = ser['possible_match']
    true_species = ser['Species'].replace('_', ' ')
    species_matches = []
    if len(lst) > 1:
        for d in lst:
            if true_species in uniprot_zf.iloc[d['index']]['Organism']: # works for both sheets
                species_matches.append(d)
        return species_matches
    elif len(lst) == 1:
        return lst
    else:
        return np.NaN
    
def filter_match(organism_match):
    """
    organism_match: a list in column organism_match
    use as matched_cisbp['sequence'] = matched_cisbp['organism_match'].apply(filter_match)
    """
    if len(organism_match) == 1:
        return uniprot_zf.iloc[organism_match[0]['index']]['Sequence']
    else:
        return np.NaN
    
def get_uniprot_id(organism_match):
    if len(organism_match) == 1:
        return uniprot_zf.iloc[organism_match[0]['index']]['UniProt ID']
    else:
        return np.NaN
    
def get_zinc_fingers(organism_match):
    if len(organism_match) == 1:
        return uniprot_zf.iloc[organism_match[0]['index']]['Zinc finger']
    else:
        return np.NaN

In [130]:
organism_match = matched_cisbp.apply(filter_by_organism, axis=1)
matched_cisbp = matched_cisbp.assign(organism_match=organism_match)
matched_cisbp = matched_cisbp.assign(num_organism_match=matched_cisbp['organism_match'].apply(len))
matched_cisbp['Sequence'] = matched_cisbp['organism_match'].apply(filter_match)
matched_cisbp['UniProt ID'] = matched_cisbp['organism_match'].apply(get_uniprot_id)
matched_cisbp['Zinc Fingers'] = matched_cisbp['organism_match'].apply(get_zinc_fingers)

In [131]:
matched_cisbp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1496 entries, 0 to 2101
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   TF                  1496 non-null   object
 1   TF Name             1496 non-null   object
 2   Gene                1496 non-null   object
 3   Motif               1496 non-null   object
 4   Species             1496 non-null   object
 5   PWM                 1496 non-null   object
 6   possible_match      1496 non-null   object
 7   num_possible_match  1496 non-null   int64 
 8   organism_match      1496 non-null   object
 9   num_organism_match  1496 non-null   int64 
 10  Sequence            1445 non-null   object
 11  UniProt ID          1445 non-null   object
 12  Zinc Fingers        1418 non-null   object
dtypes: int64(2), object(11)
memory usage: 163.6+ KB


In [137]:
final_cisbp = matched_cisbp.drop(columns=
                                 ['TF',
                                  'TF Name',
                                  'Gene',
                                  'Motif',
                                  'Species',
                                  'possible_match',
                                  'num_possible_match',
                                  'organism_match',
                                  'num_organism_match',
                                 ])
final_cisbp = final_cisbp.dropna()

In [206]:
final_cisbp.head()

,PWM,Sequence,UniProt ID,Zinc Fingers
0,"[[0.252066, 0.252066, 0.247934, 0.297521, 0.23...",MSSSYNTIALSSTPTFLLSSAAAGPGPNNFNRQEAAMTMVQQQPTS...,Q8RWX7,"ZN_FING 82..104; /note=""C2H2-type 1""; /evide..."
1,"[[0.308219, 0.25, 0.155822, 0.018836, 0.001712...",MSSSYNTIALSSTPTFLLSSAAAGPGPNNFNRQEAAMTMVQQQPTS...,Q8RWX7,"ZN_FING 82..104; /note=""C2H2-type 1""; /evide..."
6,"[[0.228068372140331, 0.23744515722474, 0.24613...",MKRDRSDYEESMKHIDIVESLMMLSRSFVVKQIDVKQSTGSKTNHN...,Q9LFG0,"ZN_FING 49..71; /note=""C2H2-type 1""; /eviden..."
7,"[[0.251653695069885, 0.289423275127517, 0.1584...",MKRDRSDYEESMKHIDIVESLMMLSRSFVVKQIDVKQSTGSKTNHN...,Q9LFG0,"ZN_FING 49..71; /note=""C2H2-type 1""; /eviden..."
13,"[[0.221000068580147, 0.674606681428864, 0.0003...",MALETLNSPTATTTARPLLRYREEMEPENLEQWAKRKRTKRQRFDH...,Q9SSW1,"ZN_FING 97..119; /note=""C2H2-type 1""; /evide..."


In [170]:
final_cisbp['Sequence'].nunique()

590

# FlyFactorSurvey

In [248]:
fpB = os.path.join('FFS_data', 'PWMfreq_DatasetB.txt')
fhB = open(fpB)
fhB_lst = (fhB.read()
           .replace('A |', '').replace('C |', '').replace('G |', '').replace('T |', '')
           .replace('\t', ' ').split('\n'))[1:-1]

for i in range(len(fhB_lst)):
    if i % 5 == 0:
        fhB_lst[i] = fhB_lst[i][1:]
    else:
        row = fhB_lst[i].strip().split()
        fhB_lst[i] = [int(elem) for elem in row]

chunks = [fhB_lst[i:(i+5)] for i in range(0, len(fhB_lst), 5)]
chunks = [[lst[0], [lst[1], lst[2], lst[3], lst[4]]] for lst in chunks]
data = []
for chunk in chunks:
    data.append({'temp_ID': chunk[0],
                 'PFM': chunk[1]})
    
FFS_PWMs = pd.DataFrame(data)
FFS_PWMs['Flybase_ID'] = FFS_PWMs['temp_ID'].apply(lambda id: id.rpartition('_')[2])

def scrape_page(page, data):
    """
    page: page being scraped
    data: add Flybase_ID, Uniprot_ID, and Domain here
    """
    page_text = requests.get(page).text
    page_important_soup = bs4.BeautifulSoup(page_text).find('tbody')
    page_results = page_important_soup.find_all('tr')
    for entry in page_results:
        Flybase_ID = entry.contents[1].contents[0].contents[0].strip()
        Uniprot_ID = entry.contents[3].contents[0].contents[0].strip()
        Domain = entry.contents[5].contents[0].strip()
        if Domain in ['zf-C2H2', 'zf-C4', 'zf-C2HC', 'zf-C3HC4', 'zf-FCS', 'zf-NF-X1']:
            data.append({'Flybase_ID': Flybase_ID,
                         'Uniprot_ID': Uniprot_ID,
                         'Domain': Domain})
            
data = []
for i in range(1, 8):
    scrape_page(f'https://mccb.umassmed.edu/ffs/BrowseData.php?PWM=1&page={i}', data)

### Merge FFS with UniProt_ZF

In [249]:
FFS_ZFs = pd.DataFrame(data)
FFS_duplicates = FFS_ZFs.merge(FFS_PWMs, on='Flybase_ID')

test_FFS = FFS_duplicates.merge(uniprot_zf, left_on = 'Uniprot_ID', right_on='UniProt ID', how='left')

In [250]:
final_FFS = (test_FFS.drop(columns=['Flybase_ID', 
                                   'Uniprot_ID',
                                   'Domain', 
                                   'temp_ID', 
                                   'Entry name', 
                                   'Status', 
                                   'Protein names', 
                                   'Gene names', 
                                   'Organism',
                                   'Length'])
             .dropna()
             .rename(columns={'PFM': 'PWM',
                              'Zinc finger': 'Zinc Fingers'})
            )
final_FFS['PWM'] = final_FFS['PWM'].apply(lambda l: np.array(l))

In [251]:
final_FFS.head()

,PWM,UniProt ID,Sequence,Zinc Fingers
0,"[[0, 10, 4, 7, 0, 0, 20, 0, 0, 11, 7, 1, 4, 9,...",Q24174,MTESTQLQTAENNNAGVVKMEPPPPATSSVSVSAAAAAHALSSLSS...,"ZN_FING 544..567; /note=""C2H2-type 1""; /evid..."
1,"[[131, 133, 112, 90, 148, 102, 190, 1, 0, 449,...",Q24174,MTESTQLQTAENNNAGVVKMEPPPPATSSVSVSAAAAAHALSSLSS...,"ZN_FING 544..567; /note=""C2H2-type 1""; /evid..."
22,"[[14, 10, 4, 2, 0, 0, 2, 0, 0, 15], [3, 0, 0, ...",Q24266,MIDAACNYLNPYAQQHQAQQQQHAQHQQHAQQQQHHLHMQQAQHHL...,"ZN_FING 333..357; /note=""C2H2-type 1""; /evid..."
34,"[[4, 4, 0, 0, 0, 1, 0, 0, 0, 0, 0], [1, 0, 0, ...",Q9VZN4,MHTDIIIGDQFAANNNYWVMQSPELDYRHELMGRLHKIEPADVELE...,"ZN_FING 664..688; /note=""C2H2-type 1""; /evid..."
35,"[[200, 248, 259, 60, 9, 10, 97, 9, 11, 12, 9, ...",Q9VZN4,MHTDIIIGDQFAANNNYWVMQSPELDYRHELMGRLHKIEPADVELE...,"ZN_FING 664..688; /note=""C2H2-type 1""; /evid..."


# JASPAR

In [191]:
fp_jaspar = os.path.join('data', 'JASPAR_data.csv')
JASPAR_data = pd.read_csv(fp_jaspar)
JASPAR_data = (JASPAR_data.drop(columns=['Unnamed: ' + str(i) for i in range(8, 23)])
              .drop(columns=['Unnamed: 0',
                             'Gene Name',
                             'Base_ID',
                             'Matrix_ID'])
              .rename(columns={'Uniprot_IDs': 'UniProt ID',
                               'zfn_sequence': 'Zinc Fingers',
                               'sequence': 'Sequence'})
              )
JASPAR_data.head()

,UniProt ID,PWM,Zinc Fingers,Sequence
0,A0A090M8Q1,"{'A': [0.06913827655310621, 0.1122244488977955...",NaN,NaN
1,A0PJY2,"{'A': [0.31590672805584435, 0.2548641021832764...","FTCEVCGKVFNAHYNLTRHMPVH, FVCKVCGKGFRQASTLCRHKI...",MDSSCHNATTKMLATAPARGNMMSTSKPLAFSIERIMARTPEPKAL...
2,A1YPR0,"{'A': [0.26745329400196655, 0.1199213630406291...","QQCPICHKVIMGAGKLPRHMRTH, YMCTICEVRFTRQDKLKIHMR...",MANDIDELIGIPFPNHSSEVLCSLNEQRHDGLLCDVLLVVQEQEYR...
3,A4QPC8,"{'A': [0.5185185185185185, 0.7407407407407407,...",NaN,NaN
4,A6NFI3,"{'A': [0.310132678743436, 0.4766802349066277, ...","TTCDVCGKVFPHRSRLAKHQRYH, FGCEECGKGFVYRSHLAIHQR...",MAALHTTPDSPAAQLERAEDGSECDPDQEEEEEEEEKGEEVQEVEE...


In [204]:
def str_to_arr(s):
    """
    s: contains the PWM_dict as contents
    """
    lst = []
    PWM_dict = ast.literal_eval(s)
    lst.append(PWM_dict['A'])
    lst.append(PWM_dict['C'])
    lst.append(PWM_dict['G'])
    lst.append(PWM_dict['T'])
    return np.array(lst)

In [209]:
JASPAR_data['PWM'] = JASPAR_data['PWM'].apply(str_to_arr)

In [212]:
JASPAR_final = JASPAR_data.dropna()

In [214]:
JASPAR_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 377 entries, 1 to 415
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   UniProt ID    377 non-null    object
 1   PWM           377 non-null    object
 2   Zinc Fingers  377 non-null    object
 3   Sequence      377 non-null    object
dtypes: object(4)
memory usage: 14.7+ KB


# Persikov

In [216]:
fp_persikov = os.path.join('data', 'Persikov 2009-14', 'database.txt')

data = []
for line in open(fp_persikov):
    lst_of_lsts = [entry.split('=') for entry in line.split()]
    temp = {}
    for lst in lst_of_lsts:
        temp[lst[0]] = lst[1]
    temp_keys = temp.keys()
    data.append(temp)

zf_df = pd.DataFrame(data)

In [217]:
zf_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4083 entries, 0 to 4082
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   source  4083 non-null   object
 1   dna     4083 non-null   object
 2   zf      4083 non-null   object
 3   f1      4083 non-null   object
 4   f2      4083 non-null   object
 5   f3      4062 non-null   object
 6   ex      4083 non-null   object
 7   Kd      660 non-null    object
 8   dna2    959 non-null    object
 9   f4      15 non-null     object
 10  f5      15 non-null     object
 11  f6      15 non-null     object
 12  KdSd    192 non-null    object
 13  Ka      320 non-null    object
 14  KaSd    124 non-null    object
 15  f12     42 non-null     object
 16  f22     42 non-null     object
 17  f32     42 non-null     object
 18  cs      16 non-null     object
 19  lbl     8 non-null      object
dtypes: object(20)
memory usage: 638.1+ KB


In [ ]:
(zf_df[(zf_df['f12'].isnull()) & 
       (zf_df['f22'].isnull()) & 
       (zf_df['f32'].isnull()) & 
       (zf_df['cs'].isnull()) & 
       (zf_df['lbl'].isnull())]
 .drop(columns=['f12', 
                'f22', 
                'f32',
                'cs',
                'lbl'])
)

## Remaining questions about handling Persikov's database
1) How should Kd and the other additional columns be handled?

2) Should we identify different proteins by their zinc fingers? (start of implementation: combine f1, f2, f3 into a single column?)

3) UniProt IDs may not serve as a practical identifier because Persikov did not include them. Instead, a better identifier could to assign 1,2,3... to each unique sequence. However, we first need to figure out the sequence... which requires looking for the scaffolds.

# Experimental Designs

In [326]:
fp_exp = os.path.join('data', 'experimental_designs.csv')
raw_exp = pd.read_csv(fp_exp).drop(columns=["Nathan: Place ZF designs mined from literature or lookup tables here. Include the path to the sequence file within this folder, and as much information as possible. If it might be relevant, include it somewhere; we can trim off what isn't needed later. Feel free to add more columns if you think of relevant features that we should include"])
raw_exp.columns = raw_exp.iloc[0]
raw_exp = raw_exp.iloc[1:].drop(columns=['Identifier', 'ACTIVITY', '# of Fingers', 'LINKER'])
raw_exp = raw_exp.drop([68,69]).loc[:89]

In [328]:
raw_exp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87 entries, 1 to 89
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Source        86 non-null     object
 1   TARGET        86 non-null     object
 2   Sequence      86 non-null     object
 3   Zinc Fingers  65 non-null     object
 4   COMMENTS      63 non-null     object
dtypes: object(5)
memory usage: 4.1+ KB


In [378]:
raw_exp = raw_exp.drop(columns=['Source', 'COMMENTS']).rename(columns={'TARGET': 'temp_DNA'})

In [384]:
raw_exp['PWM'] = raw_exp['temp_DNA'].apply(to_PWM)

In [387]:
final_exp = raw_exp.drop(columns=['temp_DNA'])

In [389]:
final_exp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87 entries, 1 to 89
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Sequence      86 non-null     object
 1   Zinc Fingers  65 non-null     object
 2   PWM           87 non-null     object
dtypes: object(3)
memory usage: 2.7+ KB


### Notes

- I haven't included all of the data mined from literature yet (probably only less than a third)
- Some rows from Database_ZF Designs may need to be dropped

# Combining the datasets

In [288]:
def PFM_to_PWM(PFM):
    """
    For turning PFMs into PWMs (assume that input is an np.array and that output should be an np.array).
    """
    if (len(PFM[0]) != len(PFM[1]) | len(PFM[1]) != len(PFM[2]) | len(PFM[2]) != len(PFM[3])):
        print(PFM, ' is a malformed matrix.')
        return None
    
    width = len(PFM[0]) 
    out = [[], [], [], []]
    for i in range(width):
        column_sum = 0 
        for letter in range(4):
            column_sum += PFM[letter][i]
        for letter in range(4):
            out[letter].append(PFM[letter][i] / column_sum)
    return np.array(out)

In [352]:
#make function for turning PWMs in np.array format into dictionary format (for portability), 
def PWM_to_dict(PWM):
    d = {}
    d['A'] = PWM[0].tolist()
    d['C'] = PWM[1].tolist()
    d['G'] = PWM[2].tolist()
    d['T'] = PWM[3].tolist()
    return d

In [362]:
def dictstr_to_PWM(dictstr):
    """
    Kind of redundant, but I wanted to give str_to_arr a better name because symmetry.
    """
    return str_to_arr(str(dictstr))

In [381]:
def to_PWM(seq):
    """
    seq: a sequence of ACGT or acgt
    Transforms s from a string to a PWM; rows are in order ACGT
    """
    if type(seq) != str:
        return np.array([[],[],[],[]])
    # temporarily, we will eliminate any letter that is not ACGTacgt
    seq_len = len(seq)
    temp = ''
    for i in range(seq_len):
        if seq[i] in ['A', 'a', 'C', 'c', 'G', 'g', 'T', 't']:
            temp += seq[i]
    
    seq_len = len(temp)
    output = [[0 for _ in range(seq_len)] for _ in range(4)]
    for i in range(seq_len):
        letter = seq[i]
        if letter in ['A', 'a']: # how to handle uppercase and lowercase letters?
            output[0][i] = 1
        elif letter in ['C', 'c']:
            output[1][i] = 1
        elif letter in ['G', 'g']:
            output[2][i] = 1
        elif letter in ['T', 't']:
            output[3][i] = 1
        #else:
            # print('Unexpected letter in ', seq)
            # return np.NaN
    return np.array(output)

### List of datasets:
1) final_cisbp
2) final_FFS
3) JASPAR_final (please remember to rename to final_JASPAR...)
4) Persikov (to be held out until some form of identification for rows can be found)
5) final_exp (see Persikov)

In [397]:
final = JASPAR_final.append(final_cisbp.append(final_FFS, ignore_index=True), ignore_index=True)

In [398]:
final.to_excel('current_standardized_data.xlsx')

In [396]:
JASPAR_final['UniProt ID'].nunique(), final_cisbp['UniProt ID'].nunique(), final_FFS['UniProt ID'].nunique()

(334, 590, 25)